# SMTM's TPS(Sep-2021) CatBoost
"SHOW ME THE MEDAL", My Team. We scored 0.81785 with ideas shared in the 'Code' and 'Discussion' (including Team GMO). In this note, We share ideas of preprocessing and models after [EDA](https://www.kaggle.com/heiswicked/smtm-s-tps-sep-2021-eda).

- Preprocessing, models based on [EDA](https://www.kaggle.com/heiswicked/smtm-s-tps-sep-2021-eda).
- Hyperparameters based on the 'codes', 'Discussion'.
- Another Try in this note for better socre.

  : Yeah, I know. Although probability of lower score than 0.81785, we continue to challenge for better.

My team used Google Colab, but writing this note for sharing. Reading this note, please consider for my writing skills in English. For these several years, I have been apart from English. By the way, I will start this note, and hope this would be helpful.

### Got errors on my Kaggle Notebooks

Right now, I faced unexpected errors with pop-ip message 'Failed to initialize Firebase client' or 'An error occurred while committing kernel: Firebase: No Firebase App '[DEFAULT]' has been created - call Firebase App.initializeApp() (app/no-app)'. These seem to be porblems of Kaggle's Backend. Don't know why the exactly reason yet.



### Trying so many times,
still the first AUC score is the best. Maybe I and my team still needs study and research speicifically on parameters and model. By the way, the first version of this note will be set as default.

These below are tries I and My tram did, which brought us lower score than 0.81782
- fold K = 5, 7 
- drop column 'n_missing' or 'miss_cnt'
- drop columns 'max_row', min_row', 'std', 'multiply' 
- CatBoostClassifier - early_stopping_rounds = 300 
- StratifiedKFold - 'random_state': 42 -> 228
- eli5 (failed because of Kaggle's error.)

Next Week or on this weekends, we will try again after more study and reasearch. 

## Import
import basic libraries first.

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn as sk

# for fast visualization
%matplotlib inline

#  retina display 
%config InlineBackend.figure_format = 'retina'

# for minus
mpl.rc('axes', unicode_minus=False)

### CatBoostClassifier
My team used CatBoosClassifier, XGBoostClassifier, LGBMClassifier. CatBoosClassifier of thses three models performed the best score. 0.81785. And then, import CatBoostClassifier and others. 
- Many teams used all of LGBM, Catboost, XGB and run code 'VottingClassifier(?)' with these models. My team is still considering about that. 

In [ ]:
# install catboost first.
!pip install catboost

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline #optional
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# from lightgbm.sklearn import LGBMClassifier
# from xgboost.sklearn import XGBClassifier
from catboost import CatBoostClassifier

## Load Data
load datas and reduce memory of datas right now.

In [ ]:
# train data
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
# test data
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
# sample solution
ss = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
# let's look into train data
print(train.shape)
train.describe()

In [ ]:
# let's look into train data
print(test.shape)
test.describe()

### Save RAM
Reducing data for saving RAM. I think this is essential in this Competition. Without this code, you have to continue to restart runtime again and again because of RAM. It doesn't matter if RAM of your PC or laptop is more than 16GB. 
- just changed name of function

In [ ]:
# source1: https://www.kaggle.com/dmitryuarov/tps-soft-voting-xgb-cb-lgbm#Basic-information
# source2: https://www.kaggle.com/rinnqd/reduce-memory-usage


def save_ram(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    print(f"After Diet : {round(end_mem, 2)}MB")
    print(f"Reduced: {round(100*(start_mem - end_mem)/(start_mem), 2)}%")

    return df

In [ ]:
train = save_ram(train)

In [ ]:
test = save_ram(test)

#### you can also check result of 'save_ram' by info method.

In [ ]:
train.info()

In [ ]:
test.info()

## New column 'mc_cat' for ML. 
We made 'miss_cnt' by counting null values in each row and made new categorical feature 'mc_cat' based on this. The column 'mc_cat' has strong relation with target column 'claim'. You can check details of this feature in [this note](https://www.kaggle.com/heiswicked/smtm-s-tps-sep-2021-eda). 

In [ ]:
train['miss_cnt'] = train.isnull().sum(axis=1)
test['miss_cnt'] = test.isnull().sum(axis=1)

In [ ]:
def mc_cat(val):
    if val == 0:
        return 0
    elif val == 1 or val == 14:
        return 1
    else:
        return 2
    
train['mc_cat'] = train['miss_cnt'].apply(mc_cat)
test['mc_cat'] = test['miss_cnt'].apply(mc_cat)

In [ ]:
train.head(2)

In [ ]:
test.head(2)

## Seprate: Numerical and Categorical and target

In [ ]:
y = train.pop('claim')
print(y.shape)

In [ ]:
X_num = train.iloc[:, 1:119]
print(X_num.shape)
X_num.head(2)

In [ ]:
test_num = test.iloc[:, 1:119]
print(test_num.shape)
test_num.head(2)

In [ ]:
X_cat1 = train['mc_cat']
X_test1 = test['mc_cat']

## Imputing null valus of numerical columns on train and test by GMO's
Imputing null values by GMO's insight. However, we need to condider result of this function below for better performance later. This is still amazing and brilliant. I and my team learn from this.

In [ ]:
# Data preprocessing functions
def get_stats_per_row(data):
    features = [x for x in data.columns.values if x[0]=="f"] 
    # n_missing is the number of null value in a row
    # Because of the same as 'miss_cnt', let 'n_missing' exist instead of 'miss_cnt'
    data['n_missing'] = data[features].isna().sum(axis=1)
    
    data['max_row'] = data[features].max(axis=1)
    data['min_row'] = data[features].min(axis=1)
    data['std'] = data[features].std(axis=1)
    # If the difference between the mean and the median was large, the median function was used.
    # If the difference is small the quantile function was used.
    # Quantile function has better than mean function.
    median_set = ['f9', 'f12', 'f26', 'f27', 'f28', 'f32', 'f33', 'f35', 'f62', 'f74', 'f82', 'f86', 'f98', 'f108', 'f116']
    for col_name in features:
        if col_name in median_set:
            data[col_name].fillna(data[col_name].median(), inplace=True)
        else:
            data[col_name].fillna(data[col_name].quantile(0.75), inplace=True)
            
    # Multiply feature is the feature that multiply all values in a row
    data['multiply'] = 1
    for feature in features:
        data['multiply'] = data[feature] * data['multiply']
    
    return data

In [ ]:
X_num = get_stats_per_row(X_num)
print(f"Count of null values in each column: {X_num.isnull().sum()}")
print(f"Count of null values in each row {X_num.isnull().sum(axis=1)}")

In [ ]:
test_num = get_stats_per_row(test_num)
print(f"Count of null values in each column: {test_num.isnull().sum()}")
print(f"Count of null values in each row {test_num.isnull().sum(axis=1)}")

## Robust Scaler
According to last [EDA note](https://www.kaggle.com/heiswicked/smtm-s-tps-sep-2021-eda). We need Robust scaling of numerical columns (X_num, test_num) for better performance and dealing with outliers.

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
# f1 ~ f118 of train data after Robust scaling
X_num = pd.DataFrame(scaler.fit_transform(X_num), index = X_num.index, columns = X_num.columns)
X_num.head(3)

In [ ]:
# f1 ~ f118 of test data after Robust scaling
test_num = pd.DataFrame(scaler.fit_transform(test_num), index = test_num.index, columns = test_num.columns)
test_num.head(3)

## Concatenation
Concatenate X_num and X_cat1 into X1.
- X_num: train's DataFrame numerical columns(f1 ~ f118) after scaling
- X_cat1 : train's categorical column 'mc_cat' without scaling
- X1 : new train data after concatenating X_num and X_cat1

Concatenate test_num and X_test into new_test.
- test_num: test's DataFrame numerical columns(f1 ~ f118) after scaling
- X_test1 : test's categorical column 'mc_cat' without scaling
- new_test : new test data after concatenating X_num and X_test1

In [ ]:
X1 = pd.concat([X_num, X_cat1], axis =1)
print(X1.shape)
X1.head(3)

In [ ]:
new_test = pd.concat([test_num, X_test1], axis =1)
print(new_test.shape)
new_test.head(3)

## del variables
for helping RAM, needs to del variables. 

In [ ]:
%who

In [ ]:
del X_num, X_cat1, test_num, X_test1, train, test

In [ ]:
# check
%who

In [ ]:
# garbage collection
import gc
gc.collect()

## Last check before running model.

In [ ]:
y

In [ ]:
X1.head(2)

In [ ]:
new_test.head(2)

## CatBoostClassifier
- GPU Accelerated         
    : CatBoost's preformance is better than LGBM, XGB and others. 
 
- Parameters.            
    : My team hasn't studied parameters of CatBoost, LGBM and XGB, which makes us to use parameter-set of others' notes considering of rest of this competition. My team tried many parameters-set of CatBoost, 'param_cat2' of these always scores the best.

In [ ]:
## source: https://www.kaggle.com/dmitryuarov/tps-soft-voting-xgb-cb-lgbm#CatBoost
## I think I wrote worng source of parameters 'param_cat2' below.
## I will find right source of parameters below, soon. 


param_cat2 = {
    'depth': 3, 
    'learning_rate': 0.014530866870832323, 
    'iterations': 44204,
    'max_bin': 265, 
    'min_data_in_leaf': 14, 
    'l2_leaf_reg': 0.004427550682515904, 
    'subsample': 0.5402586792667279, 
    'grow_policy': 'SymmetricTree', 
    'leaf_estimation_method': 'Gradient',
    'bootstrap_type': 'Bernoulli',
    'random_seed': 228,
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'task_type': 'GPU',
    'verbose': 0,
    # 'n_jobs': -1
    }

In [ ]:
model_cat = CatBoostClassifier(**param_cat2)

### cat_run_predict

There are many codes of cross-validation. My team wanna something new, which can do everything in once. As result of our struffle for this, 'can_run_predict'. Nothing special, just cross-validation and prediction into one custom function. 

- StratifiedKFold for cross-validation.           
    : Better than KFold because of keeping the ratio of target's values 0 and 1.

- predict_proba               
    : We wrote code so that model can get learned and predict(predict_proba) after each fold of cross_validation. After run all cross-validation for given K, we can get prediction datas for test data by getting mean value of predict_probas. We expect that this way can bring us predict_probas keeping apart from Over-fitting.

In [ ]:

def cat_run_predict(K, model, X, y, new_test):
    predictions = np.zeros(len(new_test))
    scores=[]
    skf = StratifiedKFold(n_splits = K, shuffle = True, random_state=42)
    for i, (train_idx, valid_idx) in enumerate(skf.split(X1, y)):
        X_train, X_test = X1.iloc[train_idx], X1.iloc[valid_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[valid_idx]

        model.fit(X_train, y_train, eval_set = [(X_test, y_test)])
        
        train_pred = model.predict_proba(X_train)[:, 1]
        train_score = roc_auc_score(y_train, train_pred)

        test_pred = model.predict_proba(X_test)[:, 1]
        test_score = roc_auc_score(y_test, test_pred)

        scores.append((train_score, test_score))
        print(f"{i+1}th Fold -ing")
        print(f"{i+1}th Fold - AUC of test data(new_test): {test_score}")
        predictions += model.predict_proba(new_test)[:,1] / K
        print(f"{i+1}th Fold - predict_proba of test data(new_test): {predictions * K / (i+1)}")

    scores_data = pd.DataFrame(scores, columns = ['train_score', 'test_score'])
    return scores_data, predictions


## K = 10

There is no specific reason for K = 10. I just checked the performances of K = 10, K =5, K = 3. Through this, I wanna know if there is somthing to consider what my team missed. 
  
- Data size is huge    
  : Data size is so huge that K = 10 could not be mattered in cross_validation. 
  
- Always got best scores at K = 10.
  : I don't know exactly reason for this. 

In [ ]:
scores_10_cat, pred_cat_10 = cat_run_predict(10, model_cat, X1, y, new_test)
print(scores_10_cat.shape)
scores_10_cat

## Check performance.
let's check the average score of test(validation data)'s AUC score('test_score') for K = 10. This is the performance of this model. 

In [ ]:
# average of scores
scores_10_cat.test_score.mean()

## Importance Features

Let's find out important features by CatBoost's method '.feature_importances_'

- The most important features.
- Not important features. 
- Visualizaton.

In [ ]:
# important features
if_list10 = pd.DataFrame( model_cat.feature_importances_, index = X1.columns, columns=['importance'])
if_list10

In [ ]:
# The most important features in this model 
if_list10.importance.sort_values(ascending=False)[:30].index

In [ ]:
# Not important features in this model 
if_list10.importance.sort_values(ascending=True)[:25].index

### Visualize feature importance 

In [ ]:
# Visualization of important features   
plt.figure(figsize = (24,12))
if_list10.importance.sort_values(ascending=False)[:30].plot.barh()
plt.title("K = 10 Catboost important features", fontsize = 28)
plt.yticks(fontsize = 14)
plt.show()

## Submission file
Let's save submission file and submit.

In [ ]:
ss['claim'] = pred_cat_10
ss.to_csv('smtm_after5_cb10.csv', index = False)
ss.head()